In [1]:
### Restart runtime after installation
! pip install -q google-cloud-speech
! pip install -q --upgrade google-cloud-translate
! pip -q install torchtext==0.6.0
! pip -q install pyvi 
! pip -q install ffmpeg-python


     |████████████████████████████████| 112kB 9.3MB/s 
     |████████████████████████████████| 102kB 8.4MB/s 
     |████████████████████████████████| 512kB 8.9MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 645kB 16.9MB/s 
     |████████████████████████████████| 102kB 6.6MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 1.2MB 7.6MB/s 
     |████████████████████████████████| 8.5MB 4.0MB/s 
     |████████████████████████████████| 747kB 53.5MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/NLP_project
! pip -q install './model/vi_spacy_model-0.2.1.tar.gz'
! python -m spacy link vi_spacy_model vi_spacy_model


/content/drive/.shortcut-targets-by-id/1kWFkPboOT_MgvjCmTkgRnw2lbnQb8HPw/NLP_project

✘ Link 'vi_spacy_model' already exists
To overwrite an existing link, use the --force flag



In [3]:
from src.transformer import *
from src.s2t import * 


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
set_key()

In [ ]:
print(opt['train_src_data'], opt['train_trg_data'])
#opt['train_src_data'] = './data/train_2/train.en'
#opt['train_trg_data'] = './data/train_2/train.vi'
#opt['n_layers'] =6 
print(opt['train_src_data'])
print(opt['train_trg_data'])
print(opt['n_layers'])

./data/train_2/train.en ./data/train_2/train.vi
./data/train_2/train.en
./data/train_2/train.vi
6


In [5]:

train_src_data, train_trg_data = read_data(opt['train_src_data'], opt['train_trg_data'])
valid_src_data, valid_trg_data = read_data(opt['valid_src_data'], opt['valid_trg_data'])

SRC, TRG = create_fields(opt['src_lang'], opt['trg_lang'])
train_iter = create_dataset(train_src_data, train_trg_data, opt['max_strlen'], opt['batchsize'], opt['device'], SRC, TRG, istrain=True)
valid_iter = create_dataset(valid_src_data, valid_trg_data, opt['max_strlen'], opt['batchsize'], opt['device'], SRC, TRG, istrain=False)

src_pad = SRC.vocab.stoi['<pad>']
trg_pad = TRG.vocab.stoi['<pad>']

model = Transformer(len(SRC.vocab), len(TRG.vocab), opt['d_model'], opt['n_layers'], opt['heads'], opt['dropout'])

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

model = model.to(opt['device'])

optimizer = ScheduledOptim(
        torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
        0.2, opt['d_model'], 4000)

criterion = LabelSmoothingLoss(len(TRG.vocab), padding_idx=trg_pad, smoothing=0.1)


loading spacy tokenizers...
creating dataset and iterator... 
creating dataset and iterator... 


In [ ]:
import pickle
f = open('SRC.pkl', 'wb')
pickle.dump(SRC, f)
f.close()

f = open('TRG.pkl', 'wb')
pickle.dump(TRG, f)
f.close()

In [ ]:
def train_precheck():
    train_dict_path = os.path.join('model','tsd.json')
    if os.path.exists(train_dict_path):
        with open(train_dict_path, 'r') as fp:
            train_state_dict = json.load(fp)
        model.load_state_dict(torch.load(train_state_dict['last_weight']))
        
    else:  
        train_state_dict = {
            'epoch' : 0,
            'bluescore' : 0,
            'last_weight' : '' 
        }
    
    return train_state_dict

In [ ]:
train_state_dict = train_precheck()
train_state_dict

{'bluescore': 0, 'epoch': 15, 'last_weight': 'model/transformer15.pth'}

In [ ]:
import time

train = 1

if train == 1:
    train_state_dict = train_precheck()
    oldbleu = train_state_dict['bluescore']
    epoch = train_state_dict['epoch']
    #epoch = 8
    bleuscore = 0
    while epoch < opt['epochs']:
        total_loss = 0
        
        for i, batch in enumerate(train_iter): 
            s = time.time()
            loss = step(model, optimizer, batch, criterion, src_pad, trg_pad)
            
            total_loss += loss
            
            if (i + 1) % opt['printevery'] == 0:
                avg_loss = total_loss/opt['printevery']
                print('epoch: {:03d} - iter: {:05d} - train loss: {:.4f} - time: {:.4f}'.format(epoch+1, i, avg_loss, time.time()- s))
                total_loss = 0

        s = time.time()
        valid_loss = validate(model, valid_iter, criterion, src_pad, trg_pad)
        
        oldbleu = bleuscore
        if epoch > 20:
            bleuscore = bleu(valid_src_data[:300], valid_trg_data[:300], model, SRC, TRG, opt['device'], opt['k'], opt['max_strlen'])
        print('epoch: {:03d} - iter: {:05d} - valid loss: {:.4f} - bleu score: {:.4f} - time: {:.4f}'.format(epoch+1, i, valid_loss, bleuscore, time.time() - s))
        
        
        if epoch % 5 == 0 and epoch >= 10:
            torch.save(model.state_dict(), os.path.join('model','transformer'+str(epoch)+'.pth'))
            train_state_dict['epoch'] = epoch
            train_state_dict['last_weight'] =os.path.join('model','transformer'+str(epoch)+'.pth')
            train_state_dict['bluescore'] = bleuscore
            with open( os.path.join('model', 'tsd.json'), 'w') as fp:
                json.dump(train_state_dict, fp, sort_keys=True, indent=4)

        if bleuscore > oldbleu:
            torch.save(model.state_dict(), os.path.join('model', 'bestaug.pth'))
        epoch += 1

epoch: 016 - iter: 00499 - train loss: 2.6733 - time: 0.2435
epoch: 016 - iter: 00999 - train loss: 2.6618 - time: 0.2332
epoch: 016 - iter: 01499 - train loss: 2.6906 - time: 0.2601
epoch: 016 - iter: 01999 - train loss: 2.7219 - time: 0.2480
epoch: 016 - iter: 02499 - train loss: 2.7433 - time: 0.2556
epoch: 016 - iter: 02999 - train loss: 2.7728 - time: 0.2421
epoch: 016 - iter: 03499 - train loss: 2.7571 - time: 0.2510
epoch: 016 - iter: 03999 - train loss: 2.7794 - time: 0.2509
epoch: 016 - iter: 04385 - valid loss: 2.3358 - bleu score: 0.0000 - time: 2.6006
epoch: 017 - iter: 00499 - train loss: 2.7696 - time: 0.2658
epoch: 017 - iter: 00999 - train loss: 2.7956 - time: 0.2693
epoch: 017 - iter: 01499 - train loss: 2.7878 - time: 0.2733
epoch: 017 - iter: 01999 - train loss: 2.8237 - time: 0.2527
epoch: 017 - iter: 02499 - train loss: 2.8246 - time: 0.2549
epoch: 017 - iter: 02999 - train loss: 2.8157 - time: 0.2645
epoch: 017 - iter: 03499 - train loss: 2.7435 - time: 0.2626
epo

/content/drive/.shortcut-targets-by-id/1kWFkPboOT_MgvjCmTkgRnw2lbnQb8HPw/NLP_project/src/transformer.py:486: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  ones = (outputs==eos_tok).nonzero() # Occurrences of end symbols for all input sentences.


epoch: 022 - iter: 04386 - valid loss: 2.2862 - bleu score: 0.2794 - time: 147.5228
epoch: 023 - iter: 00499 - train loss: 2.4934 - time: 0.2505
epoch: 023 - iter: 00999 - train loss: 2.4975 - time: 0.2732
epoch: 023 - iter: 01499 - train loss: 2.5114 - time: 0.2647
epoch: 023 - iter: 01999 - train loss: 2.5097 - time: 0.2550
epoch: 023 - iter: 02499 - train loss: 2.5306 - time: 0.2566
epoch: 023 - iter: 02999 - train loss: 2.5225 - time: 0.2560
epoch: 023 - iter: 03499 - train loss: 2.4871 - time: 0.2729
epoch: 023 - iter: 03999 - train loss: 2.4683 - time: 0.2517
epoch: 023 - iter: 04389 - valid loss: 2.2900 - bleu score: 0.2784 - time: 147.2766
epoch: 024 - iter: 00499 - train loss: 2.4638 - time: 0.2519
epoch: 024 - iter: 00999 - train loss: 2.4833 - time: 0.2544
epoch: 024 - iter: 01499 - train loss: 2.4865 - time: 0.2498
epoch: 024 - iter: 01999 - train loss: 2.4867 - time: 0.2674
epoch: 024 - iter: 02499 - train loss: 2.4859 - time: 0.2428
epoch: 024 - iter: 02999 - train loss: 

KeyboardInterrupt: ignored

In [6]:
model.load_state_dict(torch.load('./model/bestaug.pth'))

<All keys matched successfully>

In [11]:
import time
input = r_and_t()
start = time.time()
trans_sent = translate_sentence(input, model, SRC, TRG, opt['device'], opt['k'], opt['max_strlen'])
end = time.time()
response_gtrans = google_trans(input)
sent_gtrans = response_gtrans['translatedText']


print('Input: ', input)
print('Model predicted: ',trans_sent)
print('Google translated: ', sent_gtrans )
print('Time taken: {0}s with score of {1}'.format(round(end-start,4), 0))
print()

Input:  phone cases have been detected in England
Model predicted:  các trường hợp điện thoại được phát hiện ở anh
Google translated:  trường hợp điện thoại đã được phát hiện ở Anh
Time taken: 0.1926s with score of 0



In [ ]:
bleu(valid_src_data, valid_trg_data, model, SRC, TRG, opt['device'], opt['k'], opt['max_strlen'])

0.2697679102420807

In [ ]:
with open('data/tst2013.en') as f:
    train_en = f.readlines()

with open('data/tst2013.vi') as f:
    train_vi = f.readlines()



        
resu = []
for sent in train_en:
    r = google_trans(sent) 
    resu.append(r['translatedText'])


google_bleu_score(resu, train_vi)


In [ ]:
### based-line score 0.2569
#### Google scores: 0.3169

In [ ]:
mtrans = []
for sent in train_en:
    trans_sent = translate_sentence(sent, model, SRC, TRG, opt['device'], opt['k'], opt['max_strlen'])    
    mtrans.append(trans_sent)


In [ ]:

bleu_google(resu, train_vi)
